In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from os.path import expanduser, join
import pathlib

root = pathlib.Path().resolve()

In [ ]:
from os import listdir

dir = listdir(root)
if 't.ex-Graph' in dir:
  root = join(root, 't.ex-Graph')

In [ ]:
import sys
sys.path.insert(1, join(root, 'lib'))

import config
import data
import model
import export

In [ ]:
datasets = [{
    'label': 'HTTP/S Graph (SLDs)',
    'data': data.read(join(root, 'data', 'graph-data-sld.csv'))
  }, {
    'label': 'HTTP/S Graph (FQDN)',
    'data': data.read(join(root, 'data', 'graph-data-fqdn.csv'))
  }
]

In [ ]:
datasets = [{
    'label': 'HTTP/S Graph (FQDN)',
    'data': data.read(join(root, 'data', 'graph-data-fqdn.csv'))
  }
]

In [ ]:
for dataset in datasets:
  data.binary_classification_labels(config.binary_tracker, dataset.get('data'))

In [ ]:
for dataset in datasets:
  data.multi_classification_labels(config.multi_tracker, dataset.get('data'))

In [ ]:
features = data.feature_vector(datasets[0].get('data'))

In [ ]:
from sklearnex import patch_sklearn 

patch_sklearn()

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

models = {
  'continuous': [
    #LinearRegression(n_jobs=-1),
    RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1)
  ],
  'category': [
    LogisticRegression(solver='lbfgs', max_iter=1000, n_jobs=-1),
    KNeighborsClassifier(3),
    #SVC(kernel="linear", C=0.025),
    #SVC(gamma=2, C=1),
    GaussianNB(),
    DecisionTreeClassifier(),
    RandomForestClassifier(max_depth=2, random_state=0),
    AdaBoostClassifier(),
    #QuadraticDiscriminantAnalysis(),
    MLPClassifier(alpha=1, max_iter=1000)
  ]
}

In [ ]:
%matplotlib agg

results = model.compute_results(
  datasets, 
  models, 
  features, 
  [
    'weight', 
    config.binary_tracker,
    config.multi_tracker
  ]
)

In [ ]:
export.classification_results(results, root)

In [ ]:
export.aggregated_classification_results(root)

In [ ]:
"""%matplotlib inline"""

ncols = max(len(models['continuous']), len(models['category']))
export.feature_importances(2, ncols, results, root)